### Создания Feature View.

In [70]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


### Библиотека PyArrow поддерживает работу с различными форматами данных, включая Apache Arrow, Parquet, ORC и другие

In [71]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


### библиотека Python для работы с форматом файла Parquet

In [72]:
pip install fastparquet


Note: you may need to restart the kernel to use updated packages.


In [73]:
import pyarrow
print(pyarrow.__version__)



17.0.0


In [74]:
import fastparquet
print(fastparquet.__version__)


2024.5.0


### Чтение Parquet-файла с названием 'data/driver_stats.parquet'# в DataFrame df

In [75]:
import pandas as pd
df = pd.read_parquet('data/driver_stats.parquet') 

In [76]:
df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-08-16 12:00:00+00:00,1005,0.052614,0.305037,546,2024-08-31 12:51:16.842
1,2024-08-16 13:00:00+00:00,1005,0.009288,0.062068,69,2024-08-31 12:51:16.842
2,2024-08-16 14:00:00+00:00,1005,0.685283,0.282445,541,2024-08-31 12:51:16.842
3,2024-08-16 15:00:00+00:00,1005,0.765851,0.528518,256,2024-08-31 12:51:16.842
4,2024-08-16 16:00:00+00:00,1005,0.580591,0.079278,833,2024-08-31 12:51:16.842


### Feast Store Inference

In [78]:
pip install datetime

Note: you may need to restart the kernel to use updated packages.


In [79]:
pip install feast

Note: you may need to restart the kernel to use updated packages.


In [80]:
from datetime import datetime 
import pandas as pd
from feast import FeatureStore
# FeatureStore - это основной класс в библиотеке Feast, который предоставляет API для работы с хранилищем признаков (feature store). 
# Он отвечает за организацию, хранение и управление признаками (features) в вашей системе машинного обучения.

In [81]:
# Создание экземпляра FeatureStore
# '.' указывает на текущую директорию как путь к конфигурационным файлам
# store = FeatureStore('.')
# Альтернативные варианты инициализации:
# 1. С указанием пути к конфигурационным файлам
# store = FeatureStore('/path/to/config/files')

store = FeatureStore('.')

In [82]:
df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-08-16 12:00:00+00:00,1005,0.052614,0.305037,546,2024-08-31 12:51:16.842
1,2024-08-16 13:00:00+00:00,1005,0.009288,0.062068,69,2024-08-31 12:51:16.842
2,2024-08-16 14:00:00+00:00,1005,0.685283,0.282445,541,2024-08-31 12:51:16.842
3,2024-08-16 15:00:00+00:00,1005,0.765851,0.528518,256,2024-08-31 12:51:16.842
4,2024-08-16 16:00:00+00:00,1005,0.580591,0.079278,833,2024-08-31 12:51:16.842


In [83]:
# Импорт библиотек pandas и datetime
import pandas as pd
from datetime import datetime

# Создание DataFrame из словаря с данными о водителях
entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003],  # ID водителей
    "event_timestamp": [              # Временные метки событий
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8, 12, 10),
        datetime(2021, 4, 12, 16, 40, 26)
    ]
})

# Вывод созданного DataFrame
entity_df


,driver_id,event_timestamp
0,1001,2021-04-12 10:59:42
1,1002,2021-04-12 08:12:10
2,1003,2021-04-12 16:40:26


In [84]:
# Получение исторических признаков для водителей
training_df = store.get_historical_features(
    entity_df=entity_df,  # DataFrame с информацией о водителях
    features=[            # Список признаков, которые нужно получить
        'driver_hourly_stats_fresh:conv_rate',  # Конверсия в текущем часе
        'driver_hourly_stats:avg_daily_trips',   # Среднее количество поездок в день
        'driver_hourly_stats:acc_rate',          # Акцентуальность в текущем часе
    ]
).to_df()  # Преобразование результата в DataFrame


In [85]:
training_df

,driver_id,event_timestamp,conv_rate,avg_daily_trips,acc_rate
0,1001,2021-04-12 10:59:42+00:00,0.813632,771,0.627108
1,1002,2021-04-12 08:12:10+00:00,0.852087,51,0.625460
2,1003,2021-04-12 16:40:26+00:00,0.272128,360,0.853903


### Feature View - on demand

In [86]:
# Импорт необходимых библиотек
import pandas as pd
from datetime import datetime

# Создание DataFrame из словаря с данными о водителях
entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003],  # ID водителей
    "event_timestamp": [              # Временные метки событий
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8, 12, 10),
        datetime(2021, 4, 12, 16, 40, 26)
    ],
    'val_to_add': [1, 2, 3],         # Дополнительное значение для каждого водителя
    'val_to_add_2': [10, 20, 30]     # Еще одно дополнительное значение
})

# Вывод созданного DataFrame
entity_df


,driver_id,event_timestamp,val_to_add,val_to_add_2
0,1001,2021-04-12 10:59:42,1,10
1,1002,2021-04-12 08:12:10,2,20
2,1003,2021-04-12 16:40:26,3,30


In [87]:
# Получение исторических признаков для водителей
training_df = store.get_historical_features(
    entity_df=entity_df,  # DataFrame с информацией о водителях
    features=[            # Список признаков, которые нужно получить
        'driver_hourly_stats_fresh:conv_rate',      # Конверсия в текущем часе
        'driver_hourly_stats:avg_daily_trips',       # Среднее количество поездок в день
        'driver_hourly_stats:acc_rate',             # Акцентуальность в текущем часе
        'transformed_conv_rate_fresh:conv_rate_plus_val1',  # Конверсия плюс первое дополнительное значение
        'transformed_conv_rate_fresh:conv_rate_plus_val2'   # Конверсия плюс второе дополнительное значение
    ]
).to_df()  # Преобразование результата в DataFrame



In [88]:
training_df

,driver_id,event_timestamp,val_to_add,val_to_add_2,conv_rate,avg_daily_trips,acc_rate,conv_rate_plus_val1,conv_rate_plus_val2
0,1001,2021-04-12 10:59:42+00:00,1,10,0.813632,771,0.627108,1.813632,10.813632
1,1002,2021-04-12 08:12:10+00:00,2,20,0.852087,51,0.625460,2.852087,20.852087
2,1003,2021-04-12 16:40:26+00:00,3,30,0.272128,360,0.853903,3.272128,30.272128


### Спасибо за внимание!